In [1]:
import Loader
from PreProcessor import Preprocessor
from LdaMalletHandler import LdaMalletHandler
from DocumentRetriever import DocumentRetriever
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
import pandas as pd

In [2]:
import pandas as pd;
import numpy as np;
import scipy as sp;
import sklearn;
import sys;
from nltk.corpus import stopwords;
import nltk;
from gensim.models import ldamodel
import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer;
from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;
import pickle;

### Load corpus

In [3]:
corpus = Loader.load_from_folder("data")

In [4]:
len(corpus['articles'])

5741

In [5]:
corpus.keys()

dict_keys(['articles', 'dates'])

In [6]:
df = pd.DataFrame.from_dict(corpus)
df = df.drop('dates', 1)

### Pre-process and tokenize

In [7]:
# alterei o pre_processor pra devolver tokenizado e não fazer steeming (mais fácil de visualizar)
# Provavelmente teremos que fazer outro
p = Preprocessor(lang='portuguese')  
pre_processed = p.transform(corpus['articles'])

### Apply model

In [8]:
import os

if os.getcwd().split('/')[2] == 'alex':
    lda = LdaMalletHandler(mallet_path='/Users/alex/mallet-2.0.8/bin/mallet')
else:
    lda = LdaMalletHandler(mallet_path='/Users/marcosdiniz/Documents/UnB/FSI/ProjetoFinal/mallet-2.0.8/bin/mallet')
lda.run_model(model_name= "TodosFolha50", corpus=pre_processed, num_topics=50)

In [9]:
def getWords(w):
    lista = []
    for palavra in w.split('"')[:-1]:
        if '0'not in palavra:
            lista.append(palavra)
    return lista

def getTopics(model, corpus, numTopics):
    dictTopics = {}
    for i in range(numTopics):
        wAux = lda.model.print_topic(lda.ext_doc_topics(pre_processed[i])[0][0])
        words = getWords(wAux)
        dictTopics['Topic # ' + '{:02d}'.format(i+1)] = [i for i in words]
    return pd.DataFrame(dictTopics)

In [10]:
getTopics(lda.model, corpus, 50)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 41,Topic # 42,Topic # 43,Topic # 44,Topic # 45,Topic # 46,Topic # 47,Topic # 48,Topic # 49,Topic # 50
0,governo,brasil,prefeitura,rio,governo,governo,pais,brasil,brasil,sobre,...,empresa,empresa,empresa,empresa,livro,empresa,filme,milhoes,empresa,mercado
1,sobre,economia,avenida,prisao,ministro,sobre,politica,economia,economia,jornal,...,mercado,mercado,mercado,mercado,sobre,mercado,cinema,ano,mercado,cr
2,imposto,paises,local,empresario,presidente,imposto,sociedade,paises,paises,folha,...,diz,diz,diz,diz,autor,diz,filmes,mil,diz,dia
3,aumento,pais,segundo,jose,itamar,aumento,poder,pais,pais,imprensa,...,empresas,empresas,empresas,empresas,vida,empresas,teatro,passado,empresas,banco
4,impostos,mercado,cidade,policia,empresas,impostos,estado,mercado,mercado,ainda,...,produtos,produtos,produtos,produtos,mundo,produtos,historia,producao,produtos,dolar
5,estados,crescimento,prefeito,da,brasilia,estados,nao,crescimento,crescimento,brasil,...,vendas,vendas,vendas,vendas,obra,vendas,ator,cerca,vendas,juros
6,receita,comercio,centro,pc,franco,receita,povo,comercio,comercio,edicao,...,diretor,diretor,diretor,diretor,livros,diretor,diretor,maior,diretor,mes
7,empresas,setor,obras,filho,ministerio,empresas,politico,setor,setor,jornais,...,local,local,local,local,editora,local,peca,bilhoes,local,contra
8,recursos,desenvolvimento,meio,costa,sucursal,recursos,brasil,desenvolvimento,desenvolvimento,artigo,...,afirma,afirma,afirma,afirma,historia,afirma,direcao,total,afirma,dias
9,fundo,hoje,rua,pena,estatais,fundo,politicos,hoje,hoje,dia,...,ano,ano,ano,ano,homem,ano,em,em,ano,ano


# NMF

In [11]:
# colocando todos os arquivos do dataset preprocessado na lista 
# e juntando as palavras, montando o artigo pós pré processamento
dataNMF = [' '.join(text) for text in pre_processed]

In [12]:
# criando o vectorizer e contando a frequencia das palavras
vectorizer = CountVectorizer(analyzer='word', max_features=500000) # pegando as 500000 melhores features
xCounts = vectorizer.fit_transform(dataNMF)

In [13]:
# aplicando o tfidf
transformer = TfidfTransformer(smooth_idf=False)
xTfidf = transformer.fit_transform(xCounts)

In [14]:
# fazendo a normalização
xTfidfNorm = normalize(xTfidf, norm='l1', axis=1)

In [15]:
# definindo o número de tópicos
# coloquei 50 por conta de ser o mesmo númeor que estávamos usando no LDA
nTopics = 50

In [16]:
modelNMF = NMF(n_components= nTopics, init='nndsvd')

In [17]:
# treinando o modelo
modelNMF.fit(xTfidfNorm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0,
  max_iter=200, n_components=50, random_state=None, shuffle=False,
  solver='cd', tol=0.0001, verbose=0)

In [18]:
def getTopicsNMF(model, nTopWords):
    '''funcao para pegar os tópicos da NMF
    model = modelo do NMF
    nTopWord = numero de top words'''
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(nTopics):
        words_ids = model.components_[i].argsort()[:-nTopWords - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words
    
    return pd.DataFrame(word_dict)

In [19]:
getTopicsNMF(modelNMF, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 41,Topic # 42,Topic # 43,Topic # 44,Topic # 45,Topic # 46,Topic # 47,Topic # 48,Topic # 49,Topic # 50
0,telefone,revisao,us,cpi,sindicato,time,dolar,ser,policia,cr,...,hospital,bilhoes,filme,bosnia,ministro,mapa,presos,eua,empresa,enunciado
1,campos,jobim,milhoes,deputado,cruz,clube,mercado,folha,policiais,mil,...,saude,us,cinema,onu,fhc,blocos,presidio,brasil,acoes,tst
2,cep,congresso,mil,orcamento,abcd,tecnico,juros,nao,carro,xxx,...,aids,deficit,filmes,bombardeios,plano,fome,rebeliao,norte,leilao,trabalho
3,reclamante,votacoes,ano,relatorio,oswaldo,jogador,dia,diz,favela,salario,...,doenca,divida,ator,otan,governo,betinho,detentos,paises,pqu,homologacao
4,endereco,pmdb,cerca,comissao,zeze,jogadores,fechou,mas,assaltantes,ingressos,...,medicos,petroleo,atriz,servios,congresso,pib,hipodromo,americano,capital,barelli
5,limeira,relator,faturamento,pfl,crime,futebol,anterior,bem,pm,aliquota,...,medicina,arrecadacao,cineasta,feridos,fazenda,poster,penitenciaria,argentina,canhedo,homologacoes
6,rg,camara,funcionarios,subcomissao,junior,corinthians,pontos,anos,zona,reajuste,...,casos,comercial,japones,militar,fernando,miseria,celas,washington,grupo,justica
7,eliseos,constitucional,total,senador,sindicalista,santos,banco,pode,ontem,ganha,...,medico,bilhao,oscar,sarajevo,municipios,populacao,fuga,comerciais,empresas,quitacao
8,cic,revisor,recursos,parlamentares,cicero,copa,compra,tudo,carros,fonte,...,infeccao,novembro,ryoko,pessoas,estados,renda,cadeia,china,milhoes,tribunal
9,cartas,partidos,milhao,magalhaes,condutores,equipe,venda,sempre,delegado,janeiro,...,funcionarios,barris,akamatsu,aereos,henrique,colorido,ferreira,comercio,ordinarias,sindicatos
